# **Анализ важности признаков в логистической регрессии.**

Работать будем с набором данных про диабет. [Источник](https://www.kaggle.com/datasets/mathchi/diabetes-data-set)

Напомним основное уравнение прогноза $f(\vec{x}) = \frac{1}{1+exp(-\vec{x})}$

Будем полагать, что разведочный анализ данных был проведен заранее. На самом деле в контексте анализа важности признаков базовое рассмотрение распределений признаков по классам очень важно и его может быть достаточно для моделирования новых гипотез о данных и предментной области.



In [1]:
#Импорт основных библиотек

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
path = 'https://github.com/aiedu-courses/all_datasets/raw/main/diabetes.csv'
data = pd.read_csv(path)

data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


**Теория:** Тестовый набор данных отделяем ДО обучения моделей и анализа их весов. Задача тестового набора данных — независимая оценка качества модели, задача же кусочка train части — обучение и оценка весов.

In [4]:
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42) #не меняйте random_state и test_size

**Теория:** По хорошему в задаче классификации (и, на самом деле, регрессии) при разбиении на train и test следует помнить об одинаковом (дис)балансе классов. К счастью, sklearn умен и делает это за нас. Этим фактом мы также будем пользоваться при анализе устойчивости весов.

In [5]:
train_data['Outcome'].value_counts(normalize=True) #убедимся, что sklearn нам не врет

Outcome
0    0.654514
1    0.345486
Name: proportion, dtype: float64

In [6]:
test_data['Outcome'].value_counts(normalize=True)

Outcome
0    0.640625
1    0.359375
Name: proportion, dtype: float64

In [7]:
#Дальше нормализуем  данные и потом будем работать только с train data
X_train_to_cv, y_train_to_cv = train_data.drop('Outcome', axis=1), train_data['Outcome']

X_test, y_test = test_data.drop('Outcome', axis=1), test_data['Outcome']

Для стандартизации данных воспользуемся StandardScaler, который преобразовывает значение $x_i$ как :

$$x'_i = \frac{(x_i - u)}{s},$$

где $u$ — среднее значение в обучающей выборке, а s— стандартное отклонение.

При интерпретации весов модели вид проводимого преобразования **необходимо понимать**. Иначе связать измненение реального признака с изменением преобразованного не получится.

**Задание 1.**

Что пройдет с преобразованым значением $x'_i$ вектора, если мы увеличим одну из его координат?

In [8]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

#Преобразуйте данные с помощью StandardScaler

X_train_sc =
X_test_sc =

**Теория:** если вы знакомы с машинным обучением достаточно давно, то однозначно встречались с процессом *кросс-валидации.*

**Напоминание:**

Процесс кросс-валидации включает в себя следующие шаги:

1. Разделение данных: Исходный набор данных разделяется на K частей, называемых "фолдами". Обычно используется метод k-кросс-валидации, где k - это заданное заранее количество фолдов.
2. Обучение и оценка модели: Затем модель обучается K раз, каждый раз на K-1 фолде данных, а затем оценивается на оставшемся. Таким образом, мы получаем K оценок производительности модели, одну для каждого фолда.
3. Вычисление среднего значения оценок: После завершения всех итераций кросс-валидации вычисляется среднее значение оценок производительности модели. Это дает более стабильную и обобщающую оценку производительности модели. В контексте же весов мы получаем большее понимание о значимости признака на подвыборках.

Здесь мы применяем **стратифицированную кросс-валидацию**. В отличие от классической, для стратифицированной справедливо одинаковое соотношение классов в каждом фолде.

![Stratified_K_fold.png](https://ltdfoto.ru/images/2024/04/17/SNIMOK-EKRANA-2024-04-17-V-16.20.10.png)

In [29]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Создание объектов  кросс-валидации. Не меняйте random state
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=12)
kf2 = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
kf3 = StratifiedKFold(n_splits=3, shuffle=True, random_state=13)


Напишем функцию, которая будет:

1. осуществлять кросс-валидацию
2. сохранять коэффициенты регрессий на каждом фолде
3. Позволять обучать логистическую регрессию с разными силами регуляризации (гиперпараметр `C`)

In [27]:
from sklearn.metrics import f1_score

In [37]:
def train_model_and_get_coefs(model, X, y, cross_val):

  coefs = {}
  f1_scores = []


  for iter, (train_idx, val_idx) in enumerate(cross_val.split(X, y)): # для каждого фолда
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model.fit(X_train, y_train) #обучение на соответствующих X, y

    coefs[iter] = model.coef_[0] #запись коэффициентов
    f1_scores.append(f1_score(y_val, model.predict(X_val))) #запись метрики качества

  print('Mean f1:', np.mean(f1_scores))

  return model, coefs, f1_scores

In [38]:
model1 = LogisticRegression(C=1, max_iter=1000)
model2 = LogisticRegression(C=0.5, max_iter=1000)
model3 = LogisticRegression(C=0.1, max_iter=1000)

In [41]:
model1, coefs1, f1_scores1 = train_model_and_get_coefs(model1, X_train_sc, y_train_to_cv, kf)
model2, coefs2, f1_scores2 = train_model_and_get_coefs(model2, X_train_sc, y_train_to_cv, kf2)
model3, coefs3, f1_scores3 = train_model_and_get_coefs(model3, X_train_sc, y_train_to_cv, kf3)

Mean f1: 0.6322746108538904
Mean f1: 0.64030131826742
Mean f1: 0.6083921166260527


**Задание 3.** Обучите лучшую модель с различными объектами кросс-валидации. Какой получился результат?

In [1]:
#Ваш код здесь

In [2]:
#Ваш код здесь

**Задание 4.** Подумайте над получившимся результатом. Чем он плох?

Посмотрим теперь на веса, получающиеся на разных кусочках данных.

In [46]:
#Сначала соберем все коэффициенты в одну табличку
coefs1_data = pd.DataFrame(coefs1.values(), columns=[i+'_weights' for i in X_train_to_cv.columns])
coefs2_data = pd.DataFrame(coefs2.values(), columns=[i+'_weights' for i in X_train_to_cv.columns])
coefs3_data = pd.DataFrame(coefs3.values(), columns=[i+'_weights' for i in X_train_to_cv.columns])

In [49]:
coefs_data = pd.concat([coefs1_data, coefs2_data])
coefs_data = pd.concat([coefs_data, coefs3_data]).reset_index(drop=True)

coefs_data

,Pregnancies_weights,Glucose_weights,BloodPressure_weights,SkinThickness_weights,Insulin_weights,BMI_weights,DiabetesPedigreeFunction_weights,Age_weights
0,0.252373,1.287169,-0.134133,0.034789,-0.210217,0.842901,0.331671,0.469922
1,0.192848,1.114431,-0.250920,-0.015798,-0.171139,0.838305,0.043465,0.428898
2,0.169679,1.005129,-0.262357,0.122486,-0.134246,0.640522,0.168929,0.370162
3,0.233735,1.016486,-0.158494,0.119055,-0.152687,0.659973,0.179899,0.408883
4,0.059643,1.296059,-0.248057,0.061604,-0.286541,0.752629,0.135090,0.497124
5,0.309605,1.009207,-0.221791,0.018199,-0.066031,0.776479,0.198354,0.320693
6,0.098270,0.785549,-0.260500,0.010836,-0.078926,0.686686,0.136532,0.559672
7,0.296199,1.116521,-0.088824,-0.023028,0.018832,0.570893,0.214204,0.243152
8,0.181934,0.839251,-0.096053,0.129380,-0.156649,0.593549,0.127098,0.331691


**Задание 5.** Веса каких признаков имеют пересечение?

Примечание: веса назовём пересекающимися, если отрезки $[min_{f1}, max_{f1}]$ и $[min_{f2}, max_{f2}]$ имеют пересечение.

In [50]:
#Взглянем на статистики полученных весов
coefs_data.describe()

,Pregnancies_weights,Glucose_weights,BloodPressure_weights,SkinThickness_weights,Insulin_weights,BMI_weights,DiabetesPedigreeFunction_weights,Age_weights
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.199365,1.052200,-0.191237,0.050836,-0.137512,0.706882,0.170582,0.403355
std,0.084032,0.174813,0.071990,0.060095,0.088189,0.100660,0.078408,0.098156
min,0.059643,0.785549,-0.262357,-0.023028,-0.286541,0.570893,0.043465,0.243152
25%,0.169679,1.005129,-0.250920,0.010836,-0.171139,0.640522,0.135090,0.331691
50%,0.192848,1.016486,-0.221791,0.034789,-0.152687,0.686686,0.168929,0.408883
75%,0.252373,1.116521,-0.134133,0.119055,-0.078926,0.776479,0.198354,0.469922
max,0.309605,1.296059,-0.088824,0.129380,0.018832,0.842901,0.331671,0.559672


**Задание 6.** Какой признак является самым важным при всех разбиениях?

**Рассчет доверительных интервалов.**

Как видите, каждый признак "шатается" от разбиения к разбиению. Однако, некоторые значения могут быть просто связаны с шумом и случайностью. Чтобы построить какую-то более надежную оценку, чем отрезок $[min_{f_i}, max_{fi}]$ построим доверительный инетверал.

**Доверительный интервал** некоторого параметра $\theta$, это интервал, который покрывает неизвестный параметр с заданной надёжностью.

Интерпретировать, например, 95% ДИ можно так:
Если мы будем проводить аналогичное исследование на выборках одного и того же размера много раз, независимо друг от друга то с 95%-ной уверенностью мы можем утверждать, что истинное значение параметра (веса) окажется в диапазоне [w_1, w_2].

**Но 95% интервал не отражает то, что с вероятностью 0.95 истинное значение параметра лежит в найденном диапазоне!**

Для построения можно воспользоваться готовой формулой из scipy `scipy.t.interval`, но можно реализовать и руками. Сделаем оба способа, чтобы подружиться с понятиям.

Формула для вычисления доверительного интервала:
$$\hat{x}-t^*\frac{s}{\sqrt{n}}\leq μ \leq \hat{x}+t^*\frac{s}{\sqrt{n}},$$

где $μ$ искомое значение параметра, \
$t^*$ - значение [t-статистики](http://old.exponenta.ru/educat/referat/XIkonkurs/student5/tabt-st.pdf), соответствующее выбранному уровню доверия (берется из таблицы t-распределения Стьюдента), \
$s$ выборочное стандартное отклонение.

В свою очередь стандартное отклонение генеральной совокупности нам неизвестно, но мы можем его приблизить с помощью стандартного отклонения по выборке:

$s \approx \sqrt{\frac{\sum(x_i-\hat{x})^2}{n-1}}$


In [54]:
def std(x):

  #Напишите функцию для расчета стандартного отклонения

    return std


Посчитайте 95% ДИ для первого признака.

In [58]:
feature_name =  #имя 1-го признака

x = coefs_data.iloc[:, 0]
s = std(coefs_data.iloc[:, 0])

n = len(x)

In [60]:
#Посчитайте ДИ для уровнения доверия 95%. Значение t-статистики берите 2.26

low =
high =
print(f'{feature_name} ДИ ,({low}, {high})')

Pregnancies_weights ДИ ,(0.13606122979379356, 0.26266928358140607)


Теперь сверимся с библиотечным ДИ.

In [62]:
from scipy import stats as st

#Ваш код здесь

(0.13477262492592684, 0.2639578884492728)

Итеративно вычислите ДИ для всех признаков, вычисляя их длину

In [66]:

for i in range(0, 8):

    name = #имя признака
    x = coefs_data.iloc[:, i]
    interval = st.t.interval() #Ваш код здесь)

    print('Признак:', name)
    print('95% доверительный интервал:', interval)
    print('Длина ДИ:', round(interval[1]- interval[0], 2))
    print('\n')

Признак: Pregnancies_weights
95% доверительный интервал: (0.13477262492592684, 0.2639578884492728)
Длина ДИ: 0.13


Признак: Glucose_weights
95% доверительный интервал: (0.9178269493048953, 1.1865734627135813)
Длина ДИ: 0.27


Признак: BloodPressure_weights
95% доверительный интервал: (-0.24657326922960035, -0.13589986979411087)
Длина ДИ: 0.11


Признак: SkinThickness_weights
95% доверительный интервал: (0.004642670585085738, 0.09702905435247694)
Длина ДИ: 0.09


Признак: Insulin_weights
95% доверительный интервал: (-0.20529976776031456, -0.06972330638127233)
Длина ДИ: 0.14


Признак: BMI_weights
95% доверительный интервал: (0.6295078723663394, 0.7842557991145771)
Длина ДИ: 0.15


Признак: DiabetesPedigreeFunction_weights
95% доверительный интервал: (0.11031276515217801, 0.2308515995004315)
Длина ДИ: 0.12


Признак: Age_weights
95% доверительный интервал: (0.32790574453712, 0.4788048069587227)
Длина ДИ: 0.15




**Задание 7.** Доверительный интервал для какого признака получился самым широким?